In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import evaluate, print_perf 
from surprise import accuracy
from surprise import GridSearch
from stemming.porter2 import stem

import pandas as pd
import numpy as np
import json
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
import time


In [2]:
df = pd.read_csv('/Users/ting/Documents/CMPT741/project/train_rating.txt')

cols = ['train_id','user_id', 'business_id', 'rating']

df = df[cols]

review_input = '/Users/ting/Documents/CMPT741/project/train_review.json'
df_review = pd.read_json(review_input,lines=True)

In [27]:
# df_review0 = df_review.head(100000)
# from nltk.tokenize import RegexpTokenizer
# from stop_words import get_stop_words
# from nltk.stem.porter import PorterStemmer
# tokenizer = RegexpTokenizer(r'\w+')
# en_stop = get_stop_words('en')
# p_stemmer = PorterStemmer()

# for index, row in df_review0.iterrows():
#     #print(index)
#     text = row['text']
#     tokens = tokenizer.tokenize(text.lower())
#     #stopped_tokens = [i for i in tokens if not i in en_stop]
#     #texts = [p_stemmer.stem(i) for i in stopped_tokens]
#     #texts = [i for i in texts if len(i) > 1]
#     processed_text = ' '.join(tokens)
#     df_review0.loc[index,'text'] = processed_text
#df_review.iloc[1000, :].text
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from stop_words import get_stop_words

en_stop = get_stop_words('en')
p_stemmer = PorterStemmer()
def clean_review(review):
    letters_only = re.sub('[^a-zA-Z]', ' ', review)
    words = letters_only.lower().split()
    #stopwords_eng = set(stopwords.words("english"))
    useful_words = [x for x in words if not x in en_stop]
    useful_words1 = [x for x in useful_words if len(x)>1]
    texts = [p_stemmer.stem(i) for i in useful_words1]
    # Combine words into a paragraph again
    useful_words_string = ' '.join(texts)
    return(useful_words_string)

start_time = time.time()
df_review['text'] = df_review['text'].apply(clean_review)
print("--- %s seconds ---" % (time.time() - start_time)) 
#df_review.to_csv('/Users/ting/Documents/CMPT741/project/train_reviewCLEAN.csv')

--- 2352.5451061725616 seconds ---


In [46]:
#df_review.to_csv('/Users/ting/Documents/CMPT741/project/train_reviewCLEAN1.csv')
user_matrix.shape

(693209, 10)

In [35]:
df_review.rename(columns={'id': 'train_id'}, inplace=True)

In [36]:
df_join = df.join(df_review.set_index('train_id'), on = 'train_id')

#build user_document
df_user_text = df_join[['user_id','text']]
df_user_text = df_user_text.groupby('user_id').apply(lambda x: x.sum())
df_user_text.to_csv('/Users/ting/Documents/CMPT741/project/user_text.csv')

#build business_document
df_business_text = df_join[['business_id','text']]
df_business_text = df_business_text.groupby('business_id').apply(lambda x: x.sum())
df_business_text.to_csv('/Users/ting/Documents/CMPT741/project/business_text.csv')

In [39]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#df_text = pd.read_csv("/Users/ting/Documents/CMPT741/project/business_text.csv")
df_text = df_business_text
n_features = 500
corpus = df_text['text']
vectorizer = TfidfVectorizer(max_features = n_features,
                             stop_words='english')
X = vectorizer.fit_transform(corpus).toarray()
#vectorizer.get_feature_names()
business = df_text['business_id'].unique()
business_dict = {value:index for index,value in enumerate(business)}

In [40]:
df_text = df_user_text

corpus = df_text['text']
vectorizer = TfidfVectorizer(max_features = n_features,
                             stop_words='english')
U = vectorizer.fit_transform(corpus).toarray()
#vectorizer.get_feature_names()
user = df_text['user_id'].unique()
user_dict = {value:index for index,value in enumerate(user)}

In [44]:
from sklearn.decomposition import LatentDirichletAllocation
n_component = 10

lda_b = LatentDirichletAllocation(n_components=n_component, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_b.fit(X)
business_matrix = lda_b.transform(X)

lda_u = LatentDirichletAllocation(n_components=n_component, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_u.fit(U)
user_matrix = lda_u.transform(U)

# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         message = "Topic #%d: " % topic_idx
#         message += " ".join([feature_names[i]
#                              for i in topic.argsort()[:-n_top_words - 1:-1]])
#         print(message)
#     print()
# feature_names = vectorizer.get_feature_names()
# print_top_words(lda, feature_names, n_components)

In [47]:
df_test = pd.read_csv('/Users/ting/Documents/CMPT741/project/test_rating.txt')
test_cols = ['test_id','user_id', 'business_id']
df_test = df_test[test_cols]

###
feature_list = []
for i in range(n_component*2):
    n = 'feature_' + str(i)
    feature_list.append(n)

In [58]:
def buildFeatures(df):
    df_zeros_1 = pd.DataFrame(0, index=np.arange(len(df)), columns=feature_list[:n_component])
    df_zeros_2 = pd.DataFrame(0, index=np.arange(len(df)), columns=feature_list[n_component:])

    for index, row in df.iterrows():
        u_id = row['user_id']
        if u_id in user_dict:
            u_vect = user_dict[u_id]
            #df_zeros_1.loc[index]=U[u_vect]
            df_zeros_1.loc[index]=user_matrix[u_vect]

        b_id = row['business_id']
        if b_id in business_dict:
            b_vect = business_dict[b_id]
            #df_zeros_2.loc[index]=X[b_vect]
            df_zeros_2.loc[index]=business_matrix[b_vect]
    return pd.concat([df_zeros_1, df_zeros_2],axis=1)


In [57]:
# df_zeros_1 = pd.DataFrame(0, index=np.arange(len(df)), columns=feature_list[:n_component])
# df_zeros_2 = pd.DataFrame(0, index=np.arange(len(df)), columns=feature_list[n_component:])
# for index, row in df.iterrows():
#     u_id = row['user_id']
#     if u_id in user_dict:
#         u_vect = user_dict[u_id]
#         #df_zeros_1.loc[index]=U[u_vect]
#         df_zeros_1.loc[index]=business_matrix[u_vect]

#     b_id = row['business_id']
#     if b_id in business_dict:
#         b_vect = business_dict[b_id]
#         #df_zeros_2.loc[index]=X[b_vect]
#         df_zeros_2.loc[index]=user_matrix[b_vect]
business_matrix.shape

(145303, 10)

In [59]:
df_train_X = buildFeatures(df)
df_train_Y = df['rating']

df_test_X = buildFeatures(df_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_Y, 
                                                    test_size=0.33, random_state=42)
regr = RandomForestRegressor(max_depth=8, random_state=0)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
meanSquaredError = mean_squared_error(y_test, y_pred)
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)